In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import torch.optim as optim

import numpy as np
import pandas as pd
import ast
import math
import time
import random

from models.rnn.encoder_decoder_gru import EncoderRNN, DecoderRNN
from models.rnn.combined_networks import train
from rnn_utils import tensorsFromPair
from utils import showPlot, timeSince, asMinutes
from lang import load_data

%load_ext autoreload
%autoreload 2

C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


# Training Loops

In [3]:
input_seq = torch.Tensor([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]) # -> binary encoded integer sequence
target_seq = torch.Tensor([0, 2, 5, 6]) # -> character encoded function

training_pairs = [[input_seq, target_seq]]

In [4]:
eq, seq, pairs = load_data()

In [5]:
print(pairs[:2])
print(pairs[0][0])
print(pairs[0][1])

[('24,28,32,36,40,44,48,52', '4*t+4+2*8'), ('36,37,38,39,40,41,42,43', '0+5*8-5+t')]
24,28,32,36,40,44,48,52
4*t+4+2*8


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [29]:
def compare_sequences(output_sequence, target_sequence):
        magnitude: float = 0.0

        # print(output_sequence)
        # print(target_sequence)
        
        for i, value in enumerate(target_sequence):
            magnitude +=  min(5, abs(value - output_sequence[i]))#**2

        magnitude /= len(target_sequence)

        
        magnitude = min(8, magnitude)
        print(magnitude)

        return torch.tensor(magnitude)

In [30]:
from utils import eq_to_seq, is_eq_valid

def calc_magnitude(decoder_outputs, target_outputs):
    max_penalty_magnitude = torch.tensor(10)
  
    decoded_output_symbols = []
    decoded_target_symbols = []
    detached_target_outputs = target_outputs.cpu().detach().numpy().squeeze()

    for decoder_output in decoder_outputs:
        topv, topi = decoder_output.data.topk(1)
        decoded_output = eq.index2word[topi.item()]
        decoded_output_symbols.append(decoded_output)
    
    # if any(i in [0, 1] for i in decoder_outputs):
    #     print("Equation not valid 1")
    #     return max_penalty_magnitude

    for i, target_output in enumerate(detached_target_outputs):
        decoded_target = eq.index2word[target_output]
        decoded_target_symbols.append(decoded_target)
    
    stringified_output = ''.join(decoded_output_symbols)
    
    # print("--- Target ---")
    # print(detached_target_outputs)
    # print(decoded_target_symbols) 

    # print("--- Output ---")
    # print(decoder_outputs[0]) 
    # print(decoded_output_symbols) 
    # print(stringified_output)

    if is_eq_valid(stringified_output) == False:
        # print("Equation not valid 2")
        return max_penalty_magnitude

    output_sequence = eq_to_seq(stringified_output, 9)

    # if np.zeros(9) == np.array(output_sequence):
    #     print("Equation not valid 3")
    #     return max_penalty_magnitude
    # else:
    stringified_target = ''.join(decoded_target_symbols[:-1])
    # print(stringified_target)
    target_sequence = eq_to_seq(stringified_target, 9)


    
    return compare_sequences(output_sequence, target_sequence)



In [31]:
def training(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs), seq, eq)
                      for i in range(n_iters)]
    criterion =nn.NLLLoss() #  converted_loss

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, calc_magnitude)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            # enablePrint()
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            # blockPrint()

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    #showPlot(plot_losses)

In [32]:
hidden_size = 256
encoder = EncoderRNN(seq.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, eq.n_words).to(device)


# blockPrint()
training(encoder, decoder, 1000, print_every=100)

5.0
2.2222222222222223
5.0
4.555555555555555
5.0
4.666666666666667
0m 11s (- 1m 47s) (100 10%) 411.4948
5.0
5.0
5.0
5.0
4.333333333333333
0m 22s (- 1m 31s) (200 20%) 422.5660
5.0
5.0
5.0
5.0
4.333333333333333
5.0
5.0
5.0
0m 31s (- 1m 13s) (300 30%) 417.9584
5.0
3.6666666666666665
4.777777777777778
5.0
3.888888888888889
5.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
0m 39s (- 0m 59s) (400 40%) 347.1878
4.666666666666667
3.888888888888889
4.666666666666667
5.0
5.0
5.0
4.888888888888889
5.0
5.0
5.0
5.0
3.3333333333333335
5.0
0m 51s (- 0m 51s) (500 50%) 377.4268
5.0
5.0
5.0
4.333333333333333
5.0
5.0
5.0
5.0
1m 2s (- 0m 41s) (600 60%) 422.5849
4.555555555555555
5.0
5.0
2.3333333333333335
5.0
5.0
5.0
5.0
5.0
5.0
4.0
5.0
1m 12s (- 0m 31s) (700 70%) 343.8889
5.0
2.3333333333333335
4.666666666666667
4.333333333333333
5.0
4.777777777777778
4.555555555555555
5.0
1m 24s (- 0m 21s) (800 80%) 376.2519
4.444444444444445
4.888888888888889
4.333333333333333
5.0
5.0
5.0
5.0
5.0
5.0
5.0
3.5555555555555554
5.0
3

In [ ]:
from rnn_utils import tensorFromSentence

MAX_LENGTH = 10
input_lang = seq
output_lang = eq
EOS_token = 0
SOS_token = 1

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            # decoder_output, decoder_hidden, decoder_attention = decoder(
            #     decoder_input, decoder_hidden, encoder_outputs)
            # decoder_attentions[di] = decoder_attention.data

            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)  # this if or simply decoder
                
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly(encoder, decoder)